In [1]:
# %matplotlib inline
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

# IMPORT

In [1]:
from pathlib import Path
import os 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import xarray 

from pycns import CnsStream, CnsReader, get_viewer
import physio

from physio_rea_analyse import *
from custum_view import *

from configuration import *
from tools import read_events 

# READER

In [27]:
patient_id = 'HA1'
raw_folder = data_path / patient_id
cns_reader = CnsReader(raw_folder)

# PRECOMPUTE

In [28]:
cns_reader.stream_names

{'SpO2': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/SpO2,na,Numeric,Float,IntelliVue,data'),
 'CPP': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/CPP,na,Numeric,Float,IntelliVue,data'),
 'ICP': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/ICP,na,SampleSeries,Integer,IntelliVue,data'),
 'EEG_NeonatalParams': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/EEG,Composite,NeonatalParams,Composite,Amp1020,data'),
 'EtCO2': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/EtCO2,na,Numeric,Float,IntelliVue,data'),
 'ABP_Syst': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/ABP,Syst,Numeric,Float,IntelliVue,data'),
 'HR': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monitorage/raw_data/HA1/HR,na,Numeric,Float,IntelliVue,data'),
 'ECG_II': PosixPath('/crnldata/tiger/baptiste.balanca/Neuro_rea_monit

In [29]:
ecg_stream = cns_reader.streams['ECG_II']
dates = ecg_stream.get_times()
ecg, r_peaks = detect_ecg_peak(ecg_stream)
r_peaks['peak_date'] = dates[r_peaks['peak_index']]
print(r_peaks.shape)

(1273512, 3)


In [30]:
r_peaks.head()

,peak_index,peak_time,peak_date
0,207,0.416152,2021-01-02 02:31:05.879485
1,625,1.256497,2021-01-02 02:31:06.715847
2,1041,2.092822,2021-01-02 02:31:07.548208
3,1449,2.913064,2021-01-02 02:31:08.364562
4,1861,3.741347,2021-01-02 02:31:09.188919


In [31]:
co2_stream = cns_reader.streams['CO2']
dates = co2_stream.get_times()
resp_cycles = detect_resp_cycles(co2_stream)
resp_cycles['inspi_date'] = dates[resp_cycles['inspi_index']]
print(resp_cycles.shape)

(318638, 12)


In [32]:
rsa_cycles, cyclic_cardiac_rate = physio.compute_rsa(
    resp_cycles,
     r_peaks,
     srate=10.,
     two_segment=True,
     points_per_cycle=50,
)
del cyclic_cardiac_rate
rsa_cycles['date'] = resp_cycles['inspi_date']
print(rsa_cycles.shape)

(318638, 13)


In [33]:
eeg_stream = cns_reader.streams['EEG']
print(eeg_stream.channel_names)

['C3', 'C4', 'Cz', 'ECoGA1', 'ECoGA2', 'ECoGA3', 'ECoGA4', 'ECoGA5', 'ECoGA6', 'F3', 'F4', 'Fz', 'O1', 'O2', 'P3', 'P4', 'Pz', 'T7', 'T8']


In [34]:
#get dates of eeg stream 
dates = eeg_stream.get_times()
print(dates[0], dates[-1])

2021-01-02T02:34:07.892856 2021-01-14T11:41:36.299641


In [35]:
co2_stream = cns_reader.streams['CO2']

In [36]:
abp_stream = cns_reader.streams['ART']

KeyError: 'ART'

In [37]:
start_date = np.datetime64("2021-01-02T02:34:07")
stop_date = np.datetime64("2021-01-14T11:41:36")

# SOME PRETTY PLOTS

In [ ]:
# Plot the detected R peaks signal


In [ ]:
# %matplotlib inline
# fig ,ax = plt.subplots()
# spectro_bio = Spectrogram_bio(co2_stream, wsize = 300, lf = 0.1, hf = 0.4, overlap_prct = 0.5, nfft_factor = 2)
# spectro_bio.plot(ax, start_date, stop_date)


In [ ]:
# %matplotlib inline
# fig ,ax = plt.subplots()
# spectro_eeg = Spectrogram_eeg(eeg_stream, chan_name = 'ECoGA4', wsize = 10, lf=0.5, hf=45)
# spectro_eeg.plot(ax, start_date, stop_date)


In [ ]:
# %matplotlib inline
# fig ,ax = plt.subplots()
# rate_bins_resp = np.arange(5, 30, 0.5)
# resp_rate = Respi_Rate(resp_cycles, rate_bins_resp=rate_bins_resp, resp_wsize_in_mins = 5, ratio_sat = 3)
# resp_rate.plot(ax, start_date, stop_date)


In [ ]:
# %matplotlib inline
# fig ,ax = plt.subplots()
# step_bins_ecg = 1
# ecg_rate = Heart_Rate(r_peaks, step_bins_ecg =step_bins_ecg, hrv_wsize_in_mins = 2, ratio_sat = 4, plot_type = '1d')
# ecg_rate.plot(ax, start_date, stop_date)


# VIEWER

In [ ]:
# %matplotlib widget 
# viewer_raw_data = get_viewer(cns_reader, stream_names=['EEG','ABP','ICP'])
# viewer_raw_data

In [40]:
%matplotlib widget
rate_bins_resp = np.arange(5, 30, 0.5)
step_bins_ecg = 1
eeg_chan = 'ECoGA6'

ext_plots = [
            # EcgAndPeak(ecg_stream, r_peaks['peak_index']),
            Bipolar(eeg_stream, 'ECoGA6', 'ECoGA5'),
            # Filter(eeg_stream, eeg_chan, lowcut = None, highcut = 0.5),
            Filter(eeg_stream, eeg_chan, lowcut = 0.5, highcut = 40),
            AC_Envelope(eeg_stream, eeg_chan, highcut_amp=0.01),
            # Spectrogram_bio(abp_stream, wsize = 300, lf = 0.1, hf = 0.5, overlap_prct = 0.75),
            Spectrogram_eeg(eeg_stream, 'ECoGA6', wsize = 5., lf = 0.5, hf = 15, saturation_quantile=0.2),
            # RespiAndCycles(co2_stream, resp_cycles),
            Respi_Rate(resp_cycles, resp_wsize_in_mins = 5, ratio_sat = 4, rate_bins_resp=rate_bins_resp),
            Heart_Rate(r_peaks, step_bins_ecg = step_bins_ecg, hrv_wsize_in_mins = 3, ratio_sat = 3, plot_type = '2d'),
            # RSA(rsa_cycles, win_cycles_smooth = 5, n_mads_cleaning = 5),
            # Pulse_Pressure(cns_reader.streams['ABP']),
            # Spreading_depol_mono(cns_reader.streams['EEG'], down_sampling_factor = 10),
            Spreading_depol_bipol(cns_reader.streams['EEG'], down_sampling_factor = 10),
            # Spreading_depol_scalp(cns_reader.streams['EEG'], down_sampling_factor = 10),
]

viewer = get_viewer(cns_reader, stream_names=['EEG','CO2','ABP', 'ICP'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['CO2','ABP','ICP'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ICP','ART','ECG_II',], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ICP','ABP'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ABP_Mean'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['CO2'], ext_plots=ext_plots)
viewer 


Viewer(children=(VBox(children=(Canvas(footer_visible=False, header_visible=False, layout=Layout(width='100%')…

: 

In [ ]:
rate_bins_resp = np.arange(5, 25, 0.5)
step_bins_ecg = 1
eeg_chan = 'ECoGA5'

ext_plots = [
            # EcgAndPeak(ecg_stream, r_peaks['peak_index']),
            Bipolar(eeg_stream, 'ECoGA6', 'ECoGA5'),
            # Filter(eeg_stream, eeg_chan, lowcut = None, highcut = 0.5),
            Filter(eeg_stream, eeg_chan, lowcut = 0.5, highcut = 40),
            # Spectrogram_bio(abp_stream, wsize = 300, lf = 0.1, hf = 0.5, overlap_prct = 0.75),
            Spectrogram_eeg(eeg_stream, 'ECoGA5', wsize = 5., lf = 0.5, hf = 15, log_transfo=True, saturation_quantile=0.02),
            # RespiAndCycles(co2_stream, resp_cycles),
            Respi_Rate(resp_cycles, resp_wsize_in_mins = 5, ratio_sat = 4, rate_bins_resp=rate_bins_resp),
            Heart_Rate(r_peaks, step_bins_ecg = step_bins_ecg, hrv_wsize_in_mins = 3, ratio_sat = 3, plot_type = '2d'),
            RSA(rsa_cycles, win_cycles_smooth = 5, n_mads_cleaning = 5),
            Spreading_depol_bipol(cns_reader.streams['EEG'], down_sampling_factor = 10),
            ]

viewer = get_viewer(cns_reader, stream_names=['EEG','CO2','ABP', 'ICP'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['CO2','ART','ICP'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ICP','ABP','ECG_II',], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ICP','ART'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ICP','ABP'], ext_plots=ext_plots)
# viewer = get_viewer(cns_reader, stream_names=['ABP'])
viewer 

ValueError: 'ECoGA6' is not in list

# Event viewer

In [ ]:
evs = read_events(raw_folder)
evs

,Name,Description,StartTime,Duration,Target,EventDisplay,Source
0,SD,None,2022-09-09 23:56:32.490,3806.2,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User
1,2:mydriase dt,None,2022-09-10 06:00:56.000,0,Trends,"{'@color': '#8080ff', '#text': 'Line'}",CNS
2,1:propofol,None,2022-09-10 06:24:46.233,0,Trends,"{'@color': '#8080ff', '#text': 'Line'}",CNS
3,TDM,None,2022-09-10 07:56:50.279,2040.6,"EEG,Trends","{'@color': '#000000', '#text': 'Shade'}",User
4,SD,None,2022-09-10 14:34:24.651,872,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User
5,SD,None,2022-09-10 16:48:16.812,571.7,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User
6,SD,None,2022-09-10 18:50:23.445,582.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User
7,SD,None,2022-09-10 20:25:40.028,513.799,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User
8,SD,None,2022-09-10 21:24:27.666,658.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User
9,SD,None,2022-09-10 22:13:13.495,705.5,"EEG,Trends","{'@color': '#99ff99', '#text': 'Box'}",User


In [ ]:
# ev_ind = 52

# decalage_horaire = 1 # heures

# delay = np.timedelta64(decalage_horaire, 'h')
# win_jitter = np.timedelta64(1, 'h')
# time_ev = np.datetime64(evs.loc[ev_ind,'StartTime']) - delay
# new_time = time_ev - win_jitter

# print(evs.loc[ev_ind,'Name'])
# print(time_ev)
# print(evs.loc[ev_ind,'Duration'])

# viewer.time_slider.update_time(new_time=new_time)

# VIEWER MORLET

In [ ]:
%matplotlib widget
eeg_chan = 'ECoGA6'

ext_plots = [
            Monopolar(cns_reader.streams['EEG'], eeg_chan, down_sampling_factor = 3, lowcut = 0.1, highcut = 40),
            Wavelet_Power(cns_reader.streams['EEG'], eeg_chan,  f_start=3, f_stop=15, n_steps=30, n_cycles=10, amplitude_exponent=1, down_samp_compute = 2, down_samp_plot = 5, quantile_saturation = 0.01)
]

viewer = get_viewer(cns_reader, stream_names=['CO2'], ext_plots=ext_plots)
viewer 


Viewer(children=(VBox(children=(Canvas(footer_visible=False, header_visible=False, layout=Layout(width='100%')…